# ZERO-KNOWLEDGE PROOFS AND THEIR BLOCKCHAIN APPLICATIONS - GROTH16

This notebook contains the code used for Chapter 3 of my Bachelor's dissertation. The one attached to the document is slightly modified (it is simpler) to achieve more readability. We will follow the steps described in the Moon Manual and in Barreto's Pairing-Friendly Elliptic Curves of Prime Order to obtain a suitable curve

In [1]:
# Function defined to find the embedding degree of the elliptic curve of 
# order prime_order with respect to prime_searched
def embedding_degree(prime_searched,prime_order):
    order = prime_order
    found = False
    # We iterate on k from 1 to order-1 because we know that k=order-1
    # verifies the equation because of Fermat's little theorem  
    for k in range(1,prime_order-1): 
        if (prime_order^k-1)%prime_searched == 0:
            found = True
            break

    if found==False:
        return -1
    return k

In [2]:
# Define limit of iterations
max_lim = 100000
found = False

# Define the functions t(x) and p(x) as provided in the BLS6 method
def t(x):
    return x + 1

def p(x):
    return (1/3) * (1 - x)**2 * (x**2 - x + 1) + x

# Define prime value of the base prime field of the problem
q=19

# Iterate over values of x
for x in range(2, max_lim):
    p_x = p(x)
    t_x = t(x)
    expr = p_x + 1 - t_x
    # We ensure expr has q as a divisor and p_x is a prime integer and
    # embedding degree is 6
    if expr % q == 0 and p_x in ZZ and is_prime(int(p_x)) and embedding_degree(q,p_x)==6:
        print(f"Taking x = {x} satisfies the condition with p(x) = {p_x} and t(x) = {t_x}")
        found = True
        break;

if found == False:
    print(f"An x value verifying the conditions was not found. Please change the prime q")   

Taking x = 103 satisfies the condition with p(x) = 36438379 and t(x) = 104


In [3]:
print(factor(expr)),print(factor(p_x))

2^2 * 3 * 7 * 17^2 * 19 * 79
36438379


(None, None)

#### We can also check that the equation seen before holds with D=-3

In [4]:
v = var('v')
D = -3

# Define the equation
equation = 4 * p_x == t_x^2 + abs(D) * v^2

# Solve the equation
solution = solve(equation, v)
print(solution)

[
v == -6970,
v == 6970
]


#### We need to choose c_2 = quadratic residue mod 2 and c_3 = quadratic non-residue mod 3 in F_p_x

In [5]:
# Set up two random candidates following in the BLS6 method
candidate_c2 = 2
candidate_c3 = 6

# Define base field of order p_x
F_base = GF(p_x)

# Try if candidate_c2 is quadratic non-residue
c2 = F_base(candidate_c2)
try: 
    c2.nth_root(candidate_c2)
except ValueError:
    print("c2 is a quadratic non-residue")

# Try if candidate_c3 is cubic non-residue
c3 =F_base(candidate_c3)
try:
    c3.nth_root(candidate_c3)
except ValueError:
    print("c3 is a cubic non-residue") 

c2 is a quadratic non-residue
c3 is a cubic non-residue


#### We now need to check which of the polynomials generates the elliptic curve of the desired order

In [6]:
# As defined in BLS6: List of polynomials that need to be checked when j=0 and D =-3
polynomials = [[0,1],[0,c2^3],[0,c3^2],[0,c3^2*c2^3],[0,c3^(-2)],[0,c3^(-2)*c2^3]]

# Reinitialize exit variable
found = False
# Iterate on the polynomials and find which one matches the order
for poly in polynomials:
    poly_used = poly
    # Define elliptic curve
    BLS = EllipticCurve(F_base,poly)
    # Verify it matches the desired order
    if BLS.order() == expr:
        found = True
        break;

# Print the exit variable, the found polynomial and the order of the curve
print(found,poly_used, BLS, BLS.order())

True [0, 1] Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 36438379 36438276


#### Plotting the curve if q <= 13 (to avoid crashing because of the number of points)

In [7]:
if(q<=13):
    plot = BLS.plot()
    plot.show()

#### We create the q-torsion subgroups

In [8]:
# Select a randdom point in the curve that will be used to find a generator 
# of the cyclic subgroup G_1

# Set random seed for reproducibility
set_random_seed(42)

# Define the point randomly or use the one used for obtaining the results
# presented in the dissertation: P1 = BLS(25734263,28283947,1)

P1 = BLS.random_point()
P1 = BLS(25734263,28283947,1)

In [9]:
print(P1)

(25734263 : 28283947 : 1)


In [10]:
# We take the cofactor of the prime value whose cyclic group we aim to find
cofactor = factor(expr/q)
cofactor_value = cofactor.value()
cofactor, cofactor_value

(2^2 * 3 * 7 * 17^2 * 79, 1917804)

In [11]:
# We take the cofactor to obtain a generator of the cyclic group of order q
# It is an element of order q
g1 = int(cofactor_value)*P1
g1.xy()

(25136134, 25958637)

In [12]:
# We return the subgroup of order q
G1_q = [x*g1 for x in range(0,q)]
print(G1_q)

[(0 : 1 : 0), (25136134 : 25958637 : 1), (7275896 : 13973809 : 1), (13312868 : 13973809 : 1), (25099705 : 3908357 : 1), (15849615 : 22464570 : 1), (26454473 : 3908357 : 1), (14805829 : 25958637 : 1), (32934795 : 10479742 : 1), (21322580 : 3908357 : 1), (21322580 : 32530022 : 1), (32934795 : 25958637 : 1), (14805829 : 10479742 : 1), (26454473 : 32530022 : 1), (15849615 : 13973809 : 1), (25099705 : 32530022 : 1), (13312868 : 22464570 : 1), (7275896 : 22464570 : 1), (25136134 : 10479742 : 1)]


#### We verify the order of each of the elliptic cuve points

In [13]:
k = 0
for point in G1_q:
    if point.has_order(q):
        print(f"Point {point} has expected order {q}. [{k}]_1 = {point}")
    else: 
        print(f"Point {point} is the point at infinity.  Count = {k}")
    k+=1

Point (0 : 1 : 0) is the point at infinity.  Count = 0
Point (25136134 : 25958637 : 1) has expected order 19. [1]_1 = (25136134 : 25958637 : 1)
Point (7275896 : 13973809 : 1) has expected order 19. [2]_1 = (7275896 : 13973809 : 1)
Point (13312868 : 13973809 : 1) has expected order 19. [3]_1 = (13312868 : 13973809 : 1)
Point (25099705 : 3908357 : 1) has expected order 19. [4]_1 = (25099705 : 3908357 : 1)
Point (15849615 : 22464570 : 1) has expected order 19. [5]_1 = (15849615 : 22464570 : 1)
Point (26454473 : 3908357 : 1) has expected order 19. [6]_1 = (26454473 : 3908357 : 1)
Point (14805829 : 25958637 : 1) has expected order 19. [7]_1 = (14805829 : 25958637 : 1)
Point (32934795 : 10479742 : 1) has expected order 19. [8]_1 = (32934795 : 10479742 : 1)
Point (21322580 : 3908357 : 1) has expected order 19. [9]_1 = (21322580 : 3908357 : 1)
Point (21322580 : 32530022 : 1) has expected order 19. [10]_1 = (21322580 : 32530022 : 1)
Point (32934795 : 25958637 : 1) has expected order 19. [11]_1 

#### We define the base field

In [14]:
# We now use the base field and define its extension
F_p = F_base
F_pt.<t> = F_p[]

#### We find an irreducible polynomial of order the embedding degree = 6

In [15]:
# We find an irreducible polynomial of order the embedding degree k=6 to
# later define the extension of the curve
k=6
# We try first with some a polynomial to easen the representation
poly = poly = F_pt(t^6+6)
# If this polynomial is not irreducible we find a random one
while poly.is_irreducible() == False:
    poly = F_pt.random_element(degree=k)

# We return the polynomial
print(poly)

t^6 + 6


#### We define the polynomial extension field using the polynomial

In [16]:
F_pt_6.<v> = GF(p_x^k, name='v', modulus=poly)
F_pt_6.order()

2340746011669805040522667939871270637920895721

#### We define the curve extension and extract a generator of the full q-torsion group

In [17]:
# We define the curve extension
ExtBLS6 = EllipticCurve(F_pt_6,poly_used) # curve extension 
INF = ExtBLS6(0) # point at infinity 

# We find the point that is applied to itself with the Frobenius isomorphism
for P in INF.division_points(q): # full q-torsion
    if P.order() == q: # exclude point at infinity 
        piP = ExtBLS6([a.frobenius() for a in P])
        qP = p_x*P
        if piP == qP:
            break
# We define the generator of the second group of the Weil pairing domain      
g2 = ExtBLS6(P.xy())
print(g2)

(1210439*v^4 : 12881022*v^3 : 1)


#### We define the full q-torsion group

In [18]:
# We define the second cyclic group
G2_q = [ x*g2 for x in range(0,q) ]
print(G2_q)

[(0 : 1 : 0), (1210439*v^4 : 12881022*v^3 : 1), (17333865*v^4 : 35359426*v^3 : 1), (29341190*v^4 : 35359426*v^3 : 1), (24534295*v^4 : 14278112*v^3 : 1), (26201703*v^4 : 1078953*v^3 : 1), (24328005*v^4 : 14278112*v^3 : 1), (17507987*v^4 : 12881022*v^3 : 1), (17719953*v^4 : 23557357*v^3 : 1), (24014458*v^4 : 14278112*v^3 : 1), (24014458*v^4 : 22160267*v^3 : 1), (17719953*v^4 : 12881022*v^3 : 1), (17507987*v^4 : 23557357*v^3 : 1), (24328005*v^4 : 22160267*v^3 : 1), (26201703*v^4 : 35359426*v^3 : 1), (24534295*v^4 : 22160267*v^3 : 1), (29341190*v^4 : 1078953*v^3 : 1), (17333865*v^4 : 1078953*v^3 : 1), (1210439*v^4 : 23557357*v^3 : 1)]


#### We verify the weil pairing works as expected

In [19]:
# We intelligently define a set of values that we use to verify
# the Weil paining works as expected
index = [5,5,3,2] # 19 

# We verify the values are well defined considering the
# chosen prime order q
if ((index[0]*index[1] - index[2]*index[3]) % q) != 0:
   print("The product of the indices are not congruent") 

# We define the points in the respective groups and check the 
# Weil pairings return the same value
Q = ExtBLS6(G1_q[index[0]].xy())
P = ExtBLS6(G2_q[index[1]].xy())

Q_1 = ExtBLS6(G1_q[index[2]].xy())
P_1 = ExtBLS6(G2_q[index[3]].xy())

# We are meant to obtain two identical values
print(Q.weil_pairing(P,q),Q_1.weil_pairing(P_1,q))

8986105*v^5 + 6669926*v^4 + 27896706*v^3 + 13817967*v^2 + 19290815*v + 17867329 8986105*v^5 + 6669926*v^4 + 27896706*v^3 + 13817967*v^2 + 19290815*v + 17867329


### GROTH16 PROTOCOL - UNDER THE HOOD ###

In [20]:
## Including problem-specific data ##

In [21]:
# We define the base field of order q for the polynomials
# obtained in the previous program
Fq = GF(q)
Fqt.<t> = Fq[]

In [22]:
# Polynomials obtained in the previous program
# These must be changed in case the problem is changed
A = [Fqt(18*t^3 + 15*t^2 + 5*t + 10),Fqt(0),Fqt(10*t^3 + 2*t^2 + 7*t + 14),Fqt(0),Fqt(0),Fqt(0),Fqt(0),Fqt(0),Fqt(4*t^3 + 11*t^2 + 11*t + 7),Fqt(16*t^3 + 14*t + 1),Fqt(8*t^3 + 6*t^2 + 6*t + 17)]
B = [Fqt(0),Fqt(0),Fqt(0),Fqt(10*t^3 + 2*t^2 + 7*t + 14),Fqt(4*t^3 + 11*t^2 + 11*t + 7),Fqt(16*t^3 + 14*t + 1),Fqt(8*t^3 + 6*t^2 + 6*t + 17),Fqt(0),Fqt(0),Fqt(0),Fqt(0)]
C =[Fqt(0),Fqt(8*t^3 + 6*t^2 + 6*t + 17),Fqt(0),Fqt(0),Fqt(0),Fqt(0),Fqt(0),Fqt(14*t^3 + t^2 + t + 6),Fqt(10*t^3 + 2*t^2 + 7*t + 14),Fqt(4*t^3 + 11*t^2 + 11*t + 7),Fqt(16*t^3 + 14*t + 1)]


In [23]:
# Filling in the instance, witness, resulting polynomial, target polynomial
# and quotient polynomial
# These must be changed in case the problem is changed
I = [1]
W = [4,3,3,5,1,2,6,18,14]
P = Fqt(17*t^6 + 5*t^5 + 18*t^4 + 3*t^3 + 17*t^2 + 4*t + 2)
T = Fqt(t^4 + 12*t^3 + 8*t^2 + 7*t + 15)
H = Fqt(P/T)
total = [1]+ I + W

In [24]:
## Setup phase ##

In [25]:
# Initializing the common reference string
CRS = [[],[]]

In [26]:
# Setting up the simulation trapdoor
# These must be changed to test the program works for any
# sensible numeric choice
alpha = 6
beta = 5
gamma = 4
delta = 3
tau = 2

# We define the simulation trapdoor
ST = [alpha,beta,gamma,delta,tau]

# Setting up the two random values used for this execution
# These can also be changed
r = 10
s = 5

In [27]:
# We start by defining the first set of arguments of the first CRS vector
i = 0
deg = T.degree()
m = len(W)
n = len(I)

# Here is where the choices on alpha, beta and delta are first used
for g in (g1,g2):
    alpha_g = alpha * g
    beta_g = beta * g
    delta_g = delta * g
    gamma_g = gamma * g
    if  i==0:
        CRS[i].append([alpha_g,beta_g,delta_g])
    else:
        CRS[i].append([beta_g,gamma_g,delta_g])
    for j in range(0,deg):
        CRS[i][0].append(tau^j * g)
    i+=1

In [28]:
# We define empty arrays for the second, third and fourth arguments
# of the first CRS vector
second_arg = []
third_arg = []
fourth_arg = []

In [29]:
for j in range(0,n+1):
    g_1_power = g1 * ((beta * A[j](tau) + alpha * B[j](tau) + C[j](tau))/gamma)
    second_arg.append(g_1_power)

for j in range(1,m+1):
    g_1_power = g1 * ((beta * A[j+n](tau) + alpha * B[j+n](tau) + C[j+n](tau))/delta)
    third_arg.append(g_1_power)

for j in range(0,deg-1):
    g_1_power = g1 * ((tau^j * T(tau))/delta)
    fourth_arg.append(g_1_power)

In [30]:
# We append it to the CRS
CRS[0].append(second_arg)
CRS[0].append(third_arg)
CRS[0].append(fourth_arg)

In [31]:
## Proving phase ##

In [32]:
# Note that the values A,B,C,T,P,H will not be used
# Only the values found in the CRS can be used because the
# Simulation trapdoor is unkwon to the prover

# Note that the witness instance information is used to create the proof

In [33]:
# We initialize the rest of the elements that are used
# (directly or indirectly) in the CRS definition
g1_H,g1_W,g1_A,g1_B,g2_B,g1_C = 0,0,0,0,0,0

In [34]:
# Obtaining g1_W #

In [35]:
# Initializing
g1_W = 0
j=0

# Iterating on the CRS parameters and introducing the witness information
for c in CRS[0][2]:
    g1_W += W[j] * c
    j+=1
print(g1_W)

(7275896 : 22464570 : 1)


In [36]:
# Obtaining g1_H #

In [37]:
# Initializing
g1_H = 0
j = 0

# Iterating on H's coefficients introducing its information
for h in H.coefficients(sparse=False):
    g1_H += CRS[0][3][j] * h
    j+=1
print(g1_H)

(15849615 : 13973809 : 1)


In [38]:
# Obtaining g1_A #

In [39]:
# Initializing
j=tau_power=0
g1_A = CRS[0][0][0] # g1 * alpha

# Iterating on (1, instance, witness)
for iw in total:
    for a in A[j].coefficients(sparse=False):
        g1_A += CRS[0][0][3+tau_power] * a * iw
        tau_power+=1
    tau_power=0
    j+=1

# Adding last term
g1_A += r * CRS[0][0][2] # r * g1 * delta
print(g1_A)

(25099705 : 3908357 : 1)


In [40]:
# Initializing
j=tau_power=0
g1_B = CRS[0][0][1] # g1 * beta

# Iterating on (1,instance,witness)
for iw in total:
    for b in B[j].coefficients(sparse=False):
        g1_B += CRS[0][0][3+tau_power] * b * iw
        tau_power+=1
    j+=1
    tau_power=0

# Adding last term
g1_B += s * CRS[0][0][2] # s * g1 * delta
print(g1_B)

(32934795 : 10479742 : 1)


In [41]:
# Initializing
j=tau_power=0
g2_B = CRS[1][0][0] # g2 * beta

# Iterating on (1,instance,witness)
for iw in total:
    for b in B[j].coefficients(sparse=False):
        g2_B += CRS[1][0][3+tau_power] * b * iw
        tau_power+=1
    j+=1
    tau_power=0

# Adding last term
g2_B += s * CRS[1][0][2] # s * g2 * delta
g2_B

(17719953*v^4 : 23557357*v^3 : 1)

In [42]:
# Obtaining g1_C
g1_C = g1_W
g1_C += g1_H
g1_C += s * g1_A
g1_C += r * g1_B
g1_C += (-r * s) * CRS[0][0][2]
print(g1_C)

(0 : 1 : 0)


In [43]:
# The generated proof is (g1_A,g1_C,g2_B)

In [44]:
## Verifying phase ##

In [45]:
# Obtaining g1_I
j=0
g1_I = 0
I_1 = [1] + I
for c in CRS[0][1]:
    g1_I += I_1[j] * c
    j+=1
print(g1_I)

(21322580 : 32530022 : 1)


In [46]:
# Note the verifier can only use (g1_A,g1_C,g2_B), the CRS,
# the Weil pairing and the computed g1_I

In [47]:
ExtBLS6(g1_A).weil_pairing(g2_B,q) == (ExtBLS6(CRS[0][0][0]).weil_pairing(CRS[1][0][0],q) * ExtBLS6(g1_I).weil_pairing(CRS[1][0][1],q) * ExtBLS6(g1_C).weil_pairing(CRS[1][0][2],q))

True

In [48]:
##### VERIFYING THE PROTOCOL IS CORRECT #####

In [49]:
## Defining a discrete log function using the baby step giant step ##

# Input: point and cyclic prime subgroup
# Output: the discrete log of the point modulo the subgroup's order
def discrete_log(E, P, G):
    # Obtain order and ceiling m
    n = G.order() 
    m = int(n**0.5) + 1

    # Define the baby steps dictionary
    baby_steps = {}
    for j in range(m):
        baby_steps[j * G] = j

    # Define the point m in the group
    mG = m * G

    # Search for the discrete log iterating
    for i in range(m):
        giant_step = P - i * mG
        if giant_step in baby_steps:
            return i * m + baby_steps[giant_step]

    return None  # The discrete logarithm was not found

#### We can verify the evaluation of both Weil pairings are equal manually

In [50]:
discrete_log(ExtBLS6,g1_A,g1),discrete_log(ExtBLS6,g2_B,g2),discrete_log(ExtBLS6,CRS[0][0][0],g1),discrete_log(ExtBLS6,CRS[1][0][0],g2),discrete_log(ExtBLS6,g1_I,g1),discrete_log(ExtBLS6,CRS[1][0][1],g2),discrete_log(ExtBLS6,g1_C,g1),discrete_log(ExtBLS6,CRS[1][0][2],g2)

(4, 8, 6, 5, 10, 4, 0, 3)

In [51]:
((discrete_log(ExtBLS6,g1_A,g1)*discrete_log(ExtBLS6,g2_B,g2))%q) == (discrete_log(ExtBLS6,CRS[0][0][0],g1)*discrete_log(ExtBLS6,CRS[1][0][0],g2) + discrete_log(ExtBLS6,g1_I,g1)*discrete_log(ExtBLS6,CRS[1][0][1],g2) + discrete_log(ExtBLS6,g1_C,g1)*discrete_log(ExtBLS6,CRS[1][0][2],g2))%q

True

In [52]:
# If the output is True the proof has been correctly built

#### We can verify the evaluation of both Weil pairings are equal visually

In [53]:
ExtBLS6(g1_A).weil_pairing(g2_B,q), ExtBLS6(CRS[0][0][0]).weil_pairing(CRS[1][0][0],q)*ExtBLS6(g1_I).weil_pairing(CRS[1][0][1],q)*ExtBLS6(g1_C).weil_pairing(CRS[1][0][2],q)

(27452274*v^5 + 6669926*v^4 + 8541673*v^3 + 13817967*v^2 + 17147564*v + 17867329,
 27452274*v^5 + 6669926*v^4 + 8541673*v^3 + 13817967*v^2 + 17147564*v + 17867329)